In [ ]:
using CSV
using DataFrames
using Random

include("utils.jl");


In [ ]:
using ScikitLearn

@sk_import svm:SVC;
@sk_import tree:DecisionTreeClassifier;
@sk_import ensemble:VotingClassifier
@sk_import neighbors: KNeighborsClassifier;


## Read Data

In [ ]:
file_path = "dataset/super_simplified_Android_Malware.csv"

data = CSV.File(file_path, header=true) |> DataFrame;

In [ ]:
describe(data, :all)

In [ ]:
import StatsBase: countmap

columns_to_drop = ["Flow ID", " Timestamp"]
columns = names(data)

println("Size of dataframe before dropping columns $(size(data))")
for column in 1:size(data, 2)
    unique_values = countmap(data[:, column])

    if length(unique_values) == 1
        println("Adding column $(columns[column])")
        # println(unique_values)
        push!(columns_to_drop, columns[column])
    end
    
end

select!(data, Not(columns_to_drop))

println("Size of dataframe after dropping columns $(size(data))")

dropmissing!(data)

println("Size of dataframe after dropping nulls $(size(data))")

unique_data = unique(data)

println("Size of dataframe after dropping duplicating rows $(size(data))")

In [ ]:
countmap(data[:, :Label])

In [ ]:
function ip_to_decimal(ip)
    # Split the IP address into octets
    octets = split(ip, '.')
    # Convert each octet to binary and combine them into a single 32-bit number
    binary = join([string(parse(Int, octet, base=10), base=2, pad=8) for octet in octets])
    decimal = parse(Int, binary, base=2) # Convert binary to decimal
    return decimal
end

source_ips = data[!, :" Source IP"];
destination_ips = data[!, :" Destination IP"];

data[!, :"Source IP Decimal"] = map(ip -> ip_to_decimal(ip), source_ips);
data[!, :"Destination IP Decimal"] = map(ip -> ip_to_decimal(ip), destination_ips);

select!(data, Not([" Source IP", " Destination IP"]));

In [ ]:
describe(data)

In [ ]:
output_data = data[!, :Label];
select!(data, Not(:Label))
input_data = Matrix(data[!, 1:size(data, 2)]);

## First approach: binary classification

In [ ]:
function transform_binary_class(output_data)
    binary_labels = output_data .!= "Benign"
    return binary_labels
end

binary_labels = transform_binary_class(output_data)
@assert binary_labels isa BitVector
@assert input_data isa Matrix

### Preprocessing

In [ ]:
Random.seed!(42)

train_indexes, test_indexes = holdOut(size(input_data, 1), 0.2)

train_input = convert(Array{Float32, 2}, input_data[train_indexes, :])
train_binary_output = binary_labels[train_indexes]

normalizationParameters = calculateMinMaxNormalizationParameters(train_input)

normalizeMinMax!(train_input, normalizationParameters)

test_input = convert(Array{Float32, 2}, input_data[test_indexes, :])
test_binary_output = binary_labels[test_indexes]

normalizeMinMax!(test_input, normalizationParameters)

@assert size(test_input, 1) == size(test_binary_output, 1)
@assert size(train_input, 1) == size(train_binary_output, 1)

In [ ]:
Random.seed!(42)

kFolds = 10
crossValidationIndexes = crossvalidation(train_binary_output, kFolds);

In [ ]:
function generate_latex_table(metrics::Dict{String, <: Any}, final::Bool)
    
    topology = metrics["topology"]
    accuracy = metrics["accuracy"]
    recall = metrics["recall"]
    specificity = metrics["specificity"]
    f1_score = metrics["f1_score"]
    
    if final
        confusion_matrix = metrics["confusion_matrix"]
        println("$topology & $(round(accuracy*100, digits=2))\\%  & $(round(recall*100, digits=2))\\%  & $(round(specificity*100, digits=2))\\%  & $(round(f1_score*100, digits=2))\\% & $confusion_matrix \\\\")
    else
        std_accuracy = metrics["std_accuracy"]
        std_recall = metrics["std_recall"]
        std_specificity = metrics["std_specificity"]
        std_f1_score = metrics["std_f1_score"]
        println("$topology & $(round(accuracy*100, digits=2))\\% \\textit{($(round(std_accuracy, digits = 2)))} & $(round(recall*100, digits=2))\\% \\textit{($(round(std_recall, digits = 2)))} & $(round(specificity*100, digits=2))\\% \\textit{($(round(std_specificity, digits = 2)))} & $(round(f1_score*100, digits=2))\\% \\textit{($(round(std_f1_score, digits = 2)))} \\\\")
    end
    
end

### kNN

In [ ]:
include("utils.jl")
knnParameters = Dict("modelType" => :kNN, "numNeighboors" => 0)

ks = [3 , 5, 7, 10, 15, 20]
for k in ks
    knnParameters["numNeighboors"] = k
    metricsCV = (modelCrossValidation(knnParameters["modelType"], knnParameters, train_input, train_binary_output, crossValidationIndexes))
    metricsCV["topology"] = k

    generate_latex_table(metricsCV, false)
end

println("----------------------------------------------------------------")
for k in ks
    knnParameters["numNeighboors"] = k
    metrics = createAndTrainFinalModel(knnParameters["modelType"], knnParameters, train_input, train_binary_output, test_input, test_binary_output)
    metrics["topology"] = k

    generate_latex_table(metrics, true)
end

### Decision Tree

In [ ]:
include("utils.jl")
dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 1)

depths = [3, 5, 7, 10, 15, typemax(Int)]
for depth in depths
    dtParameters["maxDepth"] = depth
    metricsCV = (modelCrossValidation(dtParameters["modelType"], dtParameters, train_input, train_binary_output, crossValidationIndexes))
    metricsCV["topology"] = depth

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for depth in depths
    dtParameters["maxDepth"] = depth
    metrics = createAndTrainFinalModel(dtParameters["modelType"], dtParameters, train_input, train_binary_output, test_input, test_binary_output)
    metrics["topology"] = depth

    generate_latex_table(metrics, true)

end


### SVM

In [ ]:
include("utils.jl")
svmParameters = Dict("modelType" => :SVM, "C" => 1, "kernel" => "linear", "degree" => 3, "gamma" => "scale")

svms = [
    ("rbf", 0.1),
    ("rbf", 1.0),
    ("rbf", 10.0),
    ("poly", 0.1),
    ("poly", 1.0),
    ("linear", 0.1),
    ("linear", 1.0),
    ("linear", 10.0),
]

for (kernel, C) in svms
    svmParameters["kernel"] = kernel
    svmParameters["C"] = C
    metricsCV = (modelCrossValidation(svmParameters["modelType"], svmParameters, train_input, train_binary_output, crossValidationIndexes))
    metricsCV["topology"] = kernel * string(C)

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for (kernel, C) in svms
    svmParameters["kernel"] = kernel
    svmParameters["C"] = C
    metrics = createAndTrainFinalModel(svmParameters["modelType"], svmParameters, train_input, train_binary_output, test_input, test_binary_output)
    metrics["topology"] = kernel * string(C)

    generate_latex_table(metrics, true)

end


### ANN

In [ ]:
include("utils.jl")

#topologies = [[20], [40], [80], [100]]
topologies = [[60, 120], [80, 50], [80, 100], [100, 40]]
annParameters = Dict("modelType" => :ANN, "maxEpochs" => 200,
    "learningRate" => 0.01, "maxEpochsVal" => 30,
    "repetitions" => 30, "validationRatio" => 0.1,
    "transferFunctions" => fill(σ, 2))

for topology in topologies
    annParameters["topology"] = topology
    metricsCV = modelCrossValidation(annParameters["modelType"], annParameters, train_input, train_binary_output, crossValidationIndexes)
    metricsCV["topology"] = topology 

    generate_latex_table(metricsCV, false)
end

for topology in topologies
    annParameters["topology"] = topology
    metrics = createAndTrainFinalModel(annParameters["modelType"], annParameters, train_input, train_binary_output, test_input, test_binary_output)
    metrics["topology"] = topology 

    generate_latex_table(metrics, true)
end

### Ensembles

In [ ]:
@sk_import ensemble:StackingClassifier
@sk_import ensemble:VotingClassifier


In [ ]:
include("utils.jl")

dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 5)
knnParameters = Dict("modelType" => :kNN, "numNeighboors" => 3)
svmParameters = Dict("modelType" => :SVM, "kernel" => "rbf", "C" => 10)
Random.seed!(42)

ensemble_types = [:VotingHard, :Stacking]
final_estimators = [dtParameters, knnParameters, svmParameters]

for ensemble_type in ensemble_types
    for final_estimator in final_estimators
        metricsCV = trainClassEnsemble([:DecisionTree, :kNN, :SVM], [dtParameters, knnParameters, svmParameters], (train_input, train_binary_output), crossValidationIndexes; ensembleType = ensemble_type, final_estimator = final_estimator)
        metricsCV["topology"] = final_estimator
        generate_latex_table(metricsCV, false)

    end
end


for ensemble_type in ensemble_types
    for final_estimator in final_estimators
        metrics = createAndTrainFinalEnsemble([:DecisionTree, :kNN, :SVM], [dtParameters, knnParameters, svmParameters], (train_input, train_binary_output), (test_input, test_binary_output); ensembleType = ensemble_type, final_estimator = final_estimator)
        metrics["topology"] = final_estimator
        generate_latex_table(metrics, true)
    end
end


## Second Approach

### Data balancing

In [ ]:
using MLDataPattern;
Random.seed!(42)
X_bal, y_bal = oversample((input_data', binary_labels), shuffle = true)
X_bal = getobs(X_bal)'
y_bal = getobs(y_bal)

In [ ]:
countmap(y_bal)

In [ ]:
Random.seed!(42)

train_indexes, test_indexes = holdOut(size(X_bal, 1), 0.2)

train_input = convert(Array{Float32, 2}, X_bal[train_indexes, :])
train_balanced_binary_output = y_bal[train_indexes]

normalizationParameters = calculateMinMaxNormalizationParameters(train_input)

normalizeMinMax!(train_input, normalizationParameters)

test_input = convert(Array{Float32, 2}, X_bal[test_indexes, :])
test_balanced_binary_output = y_bal[test_indexes]

normalizeMinMax!(test_input, normalizationParameters)

@assert size(test_input, 1) == size(test_balanced_binary_output, 1)
@assert size(train_input, 1) == size(train_balanced_binary_output, 1)

In [ ]:
Random.seed!(42)

kFolds = 10
crossValidationIndexes = crossvalidation(train_balanced_binary_output, kFolds);

### kNN

In [ ]:
include("utils.jl")
knnParameters = Dict("modelType" => :kNN, "numNeighboors" => 0)

ks = [3 , 5, 7, 10, 15, 20]
for k in ks
    knnParameters["numNeighboors"] = k
    metricsCV = (modelCrossValidation(knnParameters["modelType"], knnParameters, train_input, train_balanced_binary_output, crossValidationIndexes))
    metricsCV["topology"] = k

    generate_latex_table(metricsCV, false)
end

println("----------------------------------------------------------------")
for k in ks
    knnParameters["numNeighboors"] = k
    metrics = createAndTrainFinalModel(knnParameters["modelType"], knnParameters, train_input, train_balanced_binary_output, test_input, test_balanced_binary_output)
    metrics["topology"] = k

    generate_latex_table(metrics, true)
end

### Decision Tree

In [ ]:
include("utils.jl")
dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 1)

depths = [3, 5, 7, 10, 15, nothing]
for depth in depths
    dtParameters["maxDepth"] = depth
    metricsCV = (modelCrossValidation(dtParameters["modelType"], dtParameters, train_input, train_balanced_binary_output, crossValidationIndexes))
    metricsCV["topology"] = depth

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for depth in depths
    dtParameters["maxDepth"] = depth
    metrics = createAndTrainFinalModel(dtParameters["modelType"], dtParameters, train_input, train_balanced_binary_output, test_input, test_balanced_binary_output)
    metrics["topology"] = depth

    generate_latex_table(metrics, true)

end


### SVM

In [ ]:
include("utils.jl")
svmParameters = Dict("modelType" => :SVM, "C" => 1, "kernel" => "linear", "degree" => 3, "gamma" => "scale")

svms = [
    ("rbf", 0.1),
    ("rbf", 1.0),
    ("rbf", 10.0),
    ("poly", 0.1),
    ("poly", 1.0),
    ("linear", 0.1),
    ("linear", 1.0),
    ("linear", 10.0),
]

for (kernel, C) in svms
    svmParameters["kernel"] = kernel
    svmParameters["C"] = C
    metricsCV = (modelCrossValidation(svmParameters["modelType"], svmParameters, train_input, train_balanced_binary_output, crossValidationIndexes))
    metricsCV["topology"] = kernel * " & " * string(C)

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for (kernel, C) in svms
    svmParameters["kernel"] = kernel
    svmParameters["C"] = C
    metrics = createAndTrainFinalModel(svmParameters["modelType"], svmParameters, train_input, train_balanced_binary_output, test_input, test_balanced_binary_output)
    metrics["topology"] = kernel * " & " * string(C)

    generate_latex_table(metrics, true)

end


### ANN

In [ ]:
include("utils.jl")

topologies = [[20], [40], [80], [100], [60, 120], [80, 50], [80, 100], [100, 40]]
annParameters = Dict("modelType" => :ANN, "maxEpochs" => 200,
    "learningRate" => 0.01, "maxEpochsVal" => 30,
    "repetitions" => 30, "validationRatio" => 0.1,
    "transferFunctions" => fill(σ, 2))

for topology in topologies
    annParameters["topology"] = topology
    metricsCV = modelCrossValidation(annParameters["modelType"], annParameters, train_input, train_balanced_binary_output, crossValidationIndexes)
    metricsCV["topology"] = topology 

    generate_latex_table(metricsCV, false)
end

for topology in topologies
    annParameters["topology"] = topology
    metrics = createAndTrainFinalModel(annParameters["modelType"], annParameters, train_input, train_balanced_binary_output, test_input, test_balanced_binary_output)
    metrics["topology"] = topology 

    generate_latex_table(metrics, true)
end

In [ ]:
metrics = createAndTrainFinalModel(annParameters["modelType"], annParameters, train_input, train_balanced_binary_output, test_input, test_balanced_binary_output)

### Ensembles

In [ ]:
include("utils.jl")

dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => typemax(Int))
knnParameters = Dict("modelType" => :kNN, "numNeighboors" => 3)
svmParameters = Dict("modelType" => :SVM, "kernel" => "rbf", "C" => 10)
Random.seed!(42)

ensemble_types = [:VotingHard, :Stacking]
final_estimators = [dtParameters, knnParameters, svmParameters]

for ensemble_type in ensemble_types
    for final_estimator in final_estimators
        metricsCV = trainClassEnsemble([:DecisionTree, :kNN, :SVM], [dtParameters, knnParameters, svmParameters], (train_input, train_balanced_binary_output), crossValidationIndexes; ensembleType = ensemble_type, final_estimator = final_estimator)
        metricsCV["topology"] = final_estimator
        generate_latex_table(metricsCV, false)

        if ensemble_type == :VotingHard
            break
        end

    end
end


for ensemble_type in ensemble_types
    for final_estimator in final_estimators
        metrics = createAndTrainFinalEnsemble([:DecisionTree, :kNN, :SVM], [dtParameters, knnParameters, svmParameters], (train_input, train_balanced_binary_output), (test_input, test_balanced_binary_output); ensembleType = ensemble_type, final_estimator = final_estimator)
        metrics["topology"] = final_estimator
        generate_latex_table(metrics, true)

        if ensemble_type == :VotingHard
            break
        end
    end
end


## Third approach: multiclass classification

In [ ]:
using MLDataPattern;
Random.seed!(42)
X_bal, y_bal = oversample((input_data', output_data), shuffle = true)
X_bal = getobs(X_bal)'
y_bal = getobs(y_bal)

In [ ]:
countmap(y_bal)

In [ ]:
Random.seed!(42)

train_indexes, test_indexes = holdOut(size(X_bal, 1), 0.2)

train_input = convert(Array{Float32, 2}, X_bal[train_indexes, :])
train_balanced_output = y_bal[train_indexes]

normalizationParameters = calculateMinMaxNormalizationParameters(train_input)

normalizeMinMax!(train_input, normalizationParameters)

test_input = convert(Array{Float32, 2}, X_bal[test_indexes, :])
test_balanced_output = y_bal[test_indexes]

normalizeMinMax!(test_input, normalizationParameters)

@assert size(test_input, 1) == size(test_balanced_output, 1)
@assert size(train_input, 1) == size(train_balanced_output, 1)

In [ ]:
countmap(test_balanced_output)

In [ ]:
Random.seed!(42)

kFolds = 10
crossValidationIndexes = crossvalidation(train_balanced_output, kFolds);

### kNN

In [ ]:
include("utils.jl")
knnParameters = Dict("modelType" => :kNN, "numNeighboors" => 0)

ks = [3 , 5, 7, 10, 15, 20]
for k in ks
    knnParameters["numNeighboors"] = k
    metricsCV = (modelCrossValidation(knnParameters["modelType"], knnParameters, train_input, train_balanced_output, crossValidationIndexes))
    metricsCV["topology"] = k

    generate_latex_table(metricsCV, false)
end

println("----------------------------------------------------------------")
for k in ks
    knnParameters["numNeighboors"] = k
    metrics = createAndTrainFinalModel(knnParameters["modelType"], knnParameters, train_input, train_balanced_output, test_input, test_balanced_output)
    metrics["topology"] = k

    generate_latex_table(metrics, true)
end

### Decision Tree

In [ ]:
include("utils.jl")
dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 1)

depths = [3, 5, 7, 10, 15, typemax(Int)]
for depth in depths
    dtParameters["maxDepth"] = depth
    metricsCV = (modelCrossValidation(dtParameters["modelType"], dtParameters, train_input, train_balanced_output, crossValidationIndexes))
    metricsCV["topology"] = depth

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for depth in depths
    dtParameters["maxDepth"] = depth
    metrics = createAndTrainFinalModel(dtParameters["modelType"], dtParameters, train_input, train_balanced_output, test_input, test_balanced_output)
    metrics["topology"] = depth

    generate_latex_table(metrics, true)

end


### SVM

In [ ]:
include("utils.jl")
svmParameters = Dict("modelType" => :SVM, "C" => 1, "kernel" => "linear", "degree" => 3, "gamma" => "scale")

svms = [
    ("rbf", 0.1),
    ("rbf", 1.0),
    ("rbf", 10.0),
    ("poly", 0.1),
    ("poly", 1.0),
    ("linear", 0.1),
    ("linear", 1.0),
    ("linear", 10.0),
]

for (kernel, C) in svms
    svmParameters["kernel"] = kernel
    svmParameters["C"] = C
    metricsCV = (modelCrossValidation(svmParameters["modelType"], svmParameters, train_input, train_balanced_output, crossValidationIndexes))
    metricsCV["topology"] = kernel * " & " * string(C)

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for (kernel, C) in svms
    svmParameters["kernel"] = kernel
    svmParameters["C"] = C
    metrics = createAndTrainFinalModel(svmParameters["modelType"], svmParameters, train_input, train_balanced_output, test_input, test_balanced_output)
    metrics["topology"] = kernel * " & " * string(C)

    generate_latex_table(metrics, true)

end


### ANN

In [ ]:
include("utils.jl")

topologies = [[20], [40], [80], [100], [60, 120], [80, 50], [80, 100], [100, 40]]
annParameters = Dict("modelType" => :ANN, "maxEpochs" => 200,
    "learningRate" => 0.01, "maxEpochsVal" => 30,
    "repetitions" => 30, "validationRatio" => 0.1,
    "transferFunctions" => fill(σ, 2))

for topology in topologies
    annParameters["topology"] = topology
    metricsCV = modelCrossValidation(annParameters["modelType"], annParameters, train_input, train_balanced_output, crossValidationIndexes)
    metricsCV["topology"] = topology 

    generate_latex_table(metricsCV, false)
end

for topology in topologies
    annParameters["topology"] = topology
    metrics = createAndTrainFinalModel(annParameters["modelType"], annParameters, train_input, train_balanced_output, test_input, test_balanced_output)
    metrics["topology"] = topology 

    generate_latex_table(metrics, true)
end

In [ ]:
include("utils.jl")

annParameters = Dict("modelType" => :ANN, "maxEpochs" => 1000,
    "learningRate" => 0.01, "maxEpochsVal" => 30,
    "repetitions" => 30, "validationRatio" => 0.1,
    "transferFunctions" => fill(σ, 2), "topology" => [100, 40])
metrics = createAndTrainFinalModel(annParameters["modelType"], annParameters, train_input, train_balanced_output, test_input, test_balanced_output)

In [ ]:
test_balanced_output

### Ensembles

In [ ]:
include("utils.jl")

dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => typemax(Int))
knnParameters = Dict("modelType" => :kNN, "numNeighboors" => 3)
svmParameters = Dict("modelType" => :SVM, "kernel" => "rbf", "C" => 10)
Random.seed!(42)

ensemble_types = [:VotingHard, :Stacking]
final_estimators = [dtParameters, knnParameters, svmParameters]

for ensemble_type in ensemble_types
    for final_estimator in final_estimators
        metricsCV = trainClassEnsemble([:DecisionTree, :kNN, :SVM], [dtParameters, knnParameters, svmParameters], (train_input, train_balanced_output), crossValidationIndexes; ensembleType = ensemble_type, final_estimator = final_estimator)
        metricsCV["topology"] = final_estimator
        generate_latex_table(metricsCV, false)

        if ensemble_type == :VotingHard
            break
        end

    end
end


for ensemble_type in ensemble_types
    for final_estimator in final_estimators
        metrics = createAndTrainFinalEnsemble([:DecisionTree, :kNN, :SVM], [dtParameters, knnParameters, svmParameters], (train_input, train_balanced_output), (test_input, test_balanced_output); ensembleType = ensemble_type, final_estimator = final_estimator)
        metrics["topology"] = final_estimator
        generate_latex_table(metrics, true)

        if ensemble_type == :VotingHard
            break
        end
    end
end


In [ ]:
countmap(test_balanced_output)

In [ ]:
@sk_import decomposition:PCA


In [ ]:

# pcas = 1:20:4
pca_value = 10
pca = PCA(pca_value)

#Ajust the matrix acording to the train data
fit!(pca, train_input)

#Once it is ajusted it can be used to transform the data
pca_train = pca.transform(train_input)
pca_test = pca.transform(test_input)

@assert (size(train_input)[1],pca_value) == size(pca_train)
@assert (size(test_input)[1],pca_value) == size(pca_test)

In [ ]:
size(train_input)[1],2

In [ ]:
# pcas = [2, 6, 10, 15, 20, 25, 30]

include("utils.jl")
dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 1)

depths = [3, 5, 7, 10, 15, typemax(Int)]

pca_value = 5
pca = PCA(pca_value)

#Ajust the matrix acording to the train data
fit!(pca, train_input)

#Once it is ajusted it can be used to transform the data
pca_train = pca.transform(train_input)
pca_test = pca.transform(test_input)

@assert (size(train_input)[1],pca_value) == size(pca_train)
@assert (size(test_input)[1],pca_value) == size(pca_test)


for depth in depths
    dtParameters["maxDepth"] = depth
    metricsCV = (modelCrossValidation(dtParameters["modelType"], dtParameters, pca_train, train_balanced_output, crossValidationIndexes))
    metricsCV["topology"] = depth

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for depth in depths
    dtParameters["maxDepth"] = depth
    metrics = createAndTrainFinalModel(dtParameters["modelType"], dtParameters, pca_train, train_balanced_output, pca_test, test_balanced_output)
    metrics["topology"] = depth

    generate_latex_table(metrics, true)

end
